<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/Broyden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import jax
jax.config.update('jax_enable_x64',True)
import jax.numpy as jnp
from plotly.subplots import make_subplots

In [2]:
def f(x):
    return 0.3*(x-0.9)*(x-0.3)*(x+0.5)*(x+0.8)

In [3]:
xplot = np.linspace(-1,1,100)
fig=make_subplots()
fig.add_scatter(x=xplot, y=f(xplot), mode='lines')
fig.update_layout(width=600, height=400, template='plotly_dark')

In [4]:
def newton(f, xguess, maxiter=100, tol=1e-10):

    fprime = jax.grad(f)
    x1=xguess
    for i in range(maxiter):
        x2 = x1 - f(x1)/fprime(x1)
        if jnp.abs(f(x2))<tol:
            break
        x1 = x2

    return x2

In [5]:
newton(f, 0.)

Array(0.9, dtype=float64, weak_type=True)

In [17]:
@jax.jit
def f2(x):
    return jnp.array([x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1]])

In [18]:
def newton_Nd(f, xguess, maxiter=100, tol=1e-10):

    J = jax.jit(jax.jacobian(f))
    x1=xguess
    for i in range(maxiter):
        x2 = x1 - jnp.linalg.inv(J(x1)) @ f(x1)
        print(i, f(x2))
        if jnp.linalg.norm(f(x2))<tol:
            break
        x1 = x2

    return x2

In [19]:
guess = jnp.array([0.,0.])
newton_Nd(f2, guess)

0 [ 0.5 -0.5]
1 [ 0.0859375 -0.0859375]
2 [ 0.00447052 -0.00447052]
3 [ 1.41153108e-05 -1.41153108e-05]
4 [ 1.41997303e-10 -1.41997386e-10]
5 [ 0.00000000e+00 -5.55111512e-17]


Array([0.8411639, 0.1588361], dtype=float64)

In [41]:
x1=jnp.array([0.,0.])
fval1=f2(x1)
Jinv=jnp.linalg.inv(jax.jacobian(f2)(x1))
Jinv = np.eye(2)
maxiter=500
tol=1e-10
for i in range(maxiter):
    x2 = x1 - Jinv @ fval1
    fval2 = f2(x2)
    print(x2,fval2)
    if jnp.linalg.norm(fval2)<tol:
        break
    dx = (x2-x1).reshape(-1,1)
    df = (fval2 - fval1).reshape(-1,1)
    Jinv = Jinv + (dx - Jinv@df) @ dx.T @ Jinv/ (dx.T @ Jinv @ df)
    x1 = x2
    fval1 = fval2

[1. 0.] [ 0.5 -0.5]
[0.66666667 0.33333333] [-0.31481481  0.31481481]
[0.79545455 0.20454545] [-0.10138054  0.10138054]
[0.85662834 0.14337166] [ 0.0380577 -0.0380577]
[0.83993182 0.16006818] [-0.00294674  0.00294674]
[0.8411317 0.1588683] [-7.71797779e-05  7.71797779e-05]
[0.84116397 0.15883603] [ 1.62591046e-07 -1.62591046e-07]
[0.8411639 0.1588361] [-8.94417873e-12  8.94426200e-12]


In [28]:
m_rand = np.random.uniform(size=(5,5))

In [29]:
m_rand

array([[0.95529869, 0.82886862, 0.65881543, 0.38022399, 0.15604416],
       [0.54861866, 0.46612713, 0.11187163, 0.09987144, 0.48554436],
       [0.37943167, 0.86685682, 0.3096222 , 0.10455016, 0.46828682],
       [0.6270046 , 0.23936938, 0.58897848, 0.93094904, 0.04146851],
       [0.32233232, 0.49187212, 0.92157539, 0.81335919, 0.77339369]])

In [30]:
m_rand_inv = np.linalg.inv(m_rand)

In [31]:
m_rand_inv

array([[ 0.96244423,  1.60747699, -1.60641614, -0.19259033, -0.22037321],
       [-0.61638185, -1.11845556,  2.57342372,  0.75744774, -0.77226911],
       [ 2.06129404, -1.27198971, -1.61822564, -1.79833858,  1.4589241 ],
       [-1.76678815, -0.06262142,  1.4701005 ,  2.17100365, -0.61075561],
       [-0.60725906,  1.62292949, -0.58495222, -0.54175582,  0.77986955]])

In [32]:
u=np.array([0,0,1,0,0.]).reshape(-1,1)
v = np.array([0,1.,0,0,0,]).reshape(-1,1)

In [33]:
u @ v.T

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [35]:
np.linalg.inv(m_rand + u@ v.T)

array([[ 0.68535261,  1.10468045, -0.44954539,  0.1479168 , -0.56754322],
       [-0.17249056, -0.31299271,  0.72015633,  0.21196695, -0.21611462],
       [ 1.7821654 , -1.77848253, -0.4528502 , -1.45532822,  1.10920188],
       [-1.51320969,  0.39750932,  0.41139832,  1.85939092, -0.29304539],
       [-0.70815779,  1.43984371, -0.16369517, -0.41776528,  0.65345282]])

In [37]:
m_rand_inv - m_rand_inv @ u @ v.T @ m_rand_inv / (1+v.T @ m_rand_inv @ u)

array([[ 0.68535261,  1.10468045, -0.44954539,  0.1479168 , -0.56754322],
       [-0.17249056, -0.31299271,  0.72015633,  0.21196695, -0.21611462],
       [ 1.7821654 , -1.77848253, -0.4528502 , -1.45532822,  1.10920188],
       [-1.51320969,  0.39750932,  0.41139832,  1.85939092, -0.29304539],
       [-0.70815779,  1.43984371, -0.16369517, -0.41776528,  0.65345282]])